In [1]:
import pandas as pd
from selenium import webdriver
import time as t
from bs4 import BeautifulSoup


In [2]:

def import_ibv():
    # definindo a url da b3
    url = "https://sistemaswebb3-listados.b3.com.br/indexPage/day/IBOV?language=pt-br"

    # iniciando o webdriver, que deve estar com o driver na pasta do arquivo para funcionar, nesse caso será utilizado do Chrome
    browser = webdriver.Chrome()

    # Entrando na url informada
    browser.get(url)

    # Encontrando o elemento no html para filtrarmos os dados que precisamos
    browser.find_element_by_id('segment').send_keys('Setor de Atuação')

    # Aguardando 2 segundos para a página atualizar
    t.sleep(2)

    # Agora selecionando o o menu para podermos ver todas as ações disponíveis
    browser.find_element_by_id('selectPage').send_keys('120')

    # Aguardando 1 segundo para o página atualizar
    t.sleep(1)

    # IMportando o html inteiro para o Python na variável html
    html = browser.page_source

    # Fechando o navegador
    browser.close()

    # Usando o beatiful soup e convertendo o arquivo para podermos selecionar os dados que precisamos
    bs = BeautifulSoup(html, 'html.parser')

    # Criando as listas onde iremos armazenar os dados do html
    setor, cod, acao, tipo, qtde_t, part, part_a = [], [], [], [], [], [], []

    # Usando a função find_all para selecionarmos apenas 'td' que contém os dados que usaremos e armazenando eles na variável linhas
    linhas = bs.find_all('td')

    # Criando uma variável que será usada no while loop abaixo
    i = 0

    # While loop para podermos armazenar os dados em suas respectivas colunas, onde:
    # temos 7 colunas então pegamos o primeiro elemento 0 e somando 1 até chegarmos ao último elemento para a coluna [i+6] = ao setimo elemento
    # por fim somando 7 para pularmos para a próxima linhas e seguir a lógica
    # o loop só irá para quando o número de linhas for maior do que o que os dados apresentados, para isso usados o lógica len(linhas) que nos entrega a quantia de dados
    # e como i é somado de 7 a cada iteração o loop só irá para quando todos os dados forem indexidaodos nas suas devidas colunas
    while i < (len(linhas)):
        setor.append(linhas[i].text)
        cod.append(linhas[i+1].text)
        acao.append(linhas[i+2].text)
        tipo.append(linhas[i+3].text)
        qtde_t.append(linhas[i+4].text)
        part.append(linhas[i+5].text)
        part_a.append(linhas[i+6].text)
        i += 7

    # Criando um df para armazearmos os dados
    df = pd.DataFrame({'Setor': setor[:-1],
                       'Acao': cod[:-1],
                       'Empresa': acao[:-1],
                       'Qntd_teorica': qtde_t[:-1],
                       'Part_%': part[:-1]})  # filtramos para eliminar as últimas linhas, pois a b3 entrega uma soma ao final que não queremos
    # Separando os setores e subsetores em colunas
    df['SubSetor'] = df['Setor'].apply(lambda s: s[s.rfind('/')+1:].strip())
    df['Setor'] = df['Setor'].apply(lambda s: s[:s.rfind('/')])

    # Convertendo os valores de string para int
    df['Qntd_teorica'] = df['Qntd_teorica'].apply(lambda s: s.replace(".", ""))
    df['Qntd_teorica'] = pd.to_numeric(df['Qntd_teorica'])
    df['Part_%'] = df['Part_%'].apply(lambda s: s.replace(",", ""))
    df['Part_%'] = pd.to_numeric(df['Part_%'])/1000
    df.sort_values('Part_%', ascending=False, inplace=True)

    # Criando uma colunas com a % acumulada
    df['Part_%_acum'] = df['Part_%'].cumsum()

    # Resetando o index
    df.reset_index(drop=True, inplace=True)

    # fim
    return df




In [6]:

df = import_ibv()

print(df)


                             Setor   Acao       Empresa  Qntd_teorica  Part_%  \
0                    Mats Básicos   VALE3          VALE    3843570705  16.292   
1   Petróleo, Gás e Biocombustívei  PETR4     PETROBRAS    4566442248   6.572   
2                 Financ e Outros   ITUB4  ITAUUNIBANCO    4780002924   5.793   
3                 Financ e Outros   BBDC4      BRADESCO    4691427537   4.627   
4   Petróleo, Gás e Biocombustívei  PETR3     PETROBRAS    2703343822   4.324   
..                             ...    ...           ...           ...     ...   
86                      Bens Indls  ECOR3   ECORODOVIAS     339237914   0.118   
87                 Consumo Cíclico  JHSF3     JHSF PART     305915142   0.087   
88                 Consumo Cíclico  EZTC3         EZTEC     101618236   0.078   
89                  Tec.Informação  CASH3        MELIUZ     548153725   0.052   
90                  Comput e Equip  POSI3  POSITIVO TEC      78053723   0.030   

                           

In [3]:
import yfinance as yf

In [4]:
inicio = '2022-01-01'
actio = yf.download('BBAS3.SA', start=inicio, actions=True)
actio

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume,Dividends,Stock Splits
Date,,,,,,,,
2022-01-03,29.129999,29.760000,28.799999,28.820000,27.968678,16169300,0.0,0
2022-01-04,29.120001,29.150000,28.620001,28.850000,27.997791,11714800,0.0,0
2022-01-05,28.820000,28.980000,28.200001,28.370001,27.531973,13162800,0.0,0
2022-01-06,28.420000,28.860001,28.200001,28.600000,27.755177,10135500,0.0,0
2022-01-07,28.610001,28.820000,28.400000,28.629999,27.784290,8188700,0.0,0
...,...,...,...,...,...,...,...,...
2022-04-12,35.209999,35.490002,34.860001,35.000000,35.000000,12418700,0.0,0
2022-04-13,35.410000,35.450001,34.900002,35.169998,35.169998,12807900,0.0,0
2022-04-14,35.090000,35.540001,35.070000,35.200001,35.200001,10692100,0.0,0


In [5]:
actio[actio['Dividends']!=0]

,Open,High,Low,Close,Adj Close,Volume,Dividends,Stock Splits
Date,,,,,,,,
2022-03-03,34.849998,35.230000,34.560001,34.770000,34.552235,12833600,0.824721,0
2022-03-15,33.410000,33.450001,32.759998,33.040001,33.040001,10441900,0.210628,0


In [6]:
div = actio[actio['Dividends'] != 0]

In [9]:
div

,Open,High,Low,Close,Adj Close,Volume,Dividends,Stock Splits
Date,,,,,,,,
2022-03-03,34.849998,35.230000,34.560001,34.770000,34.552235,12833600,0.824721,0
2022-03-15,33.410000,33.450001,32.759998,33.040001,33.040001,10441900,0.210628,0


In [10]:
div['test'] = div['Close'] - div['Adj Close']

/tmp/ipykernel_37088/3025021820.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  div['test'] = div['Close'] - div['Adj Close']


In [11]:
div

,Open,High,Low,Close,Adj Close,Volume,Dividends,Stock Splits,test
Date,,,,,,,,,
2022-03-03,34.849998,35.230000,34.560001,34.770000,34.552235,12833600,0.824721,0,0.217766
2022-03-15,33.410000,33.450001,32.759998,33.040001,33.040001,10441900,0.210628,0,0.000000


In [15]:
actio[actio.Close != actio['Adj Close']]

,Open,High,Low,Close,Adj Close,Volume,Dividends,Stock Splits
Date,,,,,,,,
2022-01-03,29.129999,29.760000,28.799999,28.820000,27.968678,16169300,0.000000,0
2022-01-04,29.120001,29.150000,28.620001,28.850000,27.997791,11714800,0.000000,0
2022-01-05,28.820000,28.980000,28.200001,28.370001,27.531973,13162800,0.000000,0
2022-01-06,28.420000,28.860001,28.200001,28.600000,27.755177,10135500,0.000000,0
2022-01-07,28.610001,28.820000,28.400000,28.629999,27.784290,8188700,0.000000,0
2022-01-10,28.540001,29.090000,28.450001,28.780001,27.929861,9173700,0.000000,0
2022-01-11,28.719999,29.020000,28.680000,28.900000,28.046314,12649200,0.000000,0
2022-01-12,28.920000,29.360001,28.850000,29.180000,28.318045,14166100,0.000000,0
2022-01-13,29.059999,30.020000,29.059999,29.650000,28.774160,17158100,0.000000,0
